In [1]:
import pandas as pd
import numpy as np
import functools

In [2]:
# Read in the training dataset provided by Optiver
df_raw = pd.read_csv('train.csv')

# Read in the stock returns calculated previously
stock_rets = pd.read_parquet('stock_rets.gzip')

# Read in the index returns estimated by PCA
all_idx_rets = pd.read_parquet('all_idx_rets.gzip')

In [15]:
# Merge all of the data in one dataframe
df_trim = df_raw.drop(columns=['reference_price', 'bid_price', 'ask_price', 'wap', 'far_price', 'near_price', 'time_id', 'row_id'])
df_temp = pd.merge(left=df_trim, right=stock_rets, how='left', on=['date_id', 'seconds_in_bucket', 'stock_id']) 
df_all = pd.merge(left=df_temp, right=all_idx_rets, how='left', on=['date_id', 'seconds_in_bucket'])

In [16]:
# Calculate the active stock returns (reference, wap and mid-based) from the data
df_active = (
    df_all
    .assign(
        active_ref_return = lambda x: x['ref_return'] - x['idx_ref_return'],
        active_wap_return = lambda x: x['wap_return'] - x['idx_wap_return'],
        active_mid_return = lambda x: x['mid_return'] - x['idx_mid_return'],
        order_book_imbalance = lambda x: (x['bid_size'] - x['ask_size']) / (x['bid_size'] + x['ask_size']),
        auction_matched_pct = lambda x: x['matched_size'] / (x['imbalance_size'] + x['matched_size']),
    )
)

# Remove any unnecessary columns
df_active.drop(columns=['imbalance_size', 'matched_size', 'bid_size', 'ask_size', 'ref_return', 'wap_return', 'mid_return', 'idx_ref_return', 'idx_wap_return', 'idx_mid_return'], inplace=True)

In [17]:
# Create EWMA versions of the returns and imbalance measure using different half lives

df_ewm = (
    df_active
    .assign(
        **{
            f"{col}_{halflife}": functools.partial(
                lambda x, c, h: x.groupby(['date_id', 'stock_id'])[c].transform(lambda s: s.ewm(halflife=h).mean()),
                h = halflife,
                c=col
            )
            for col in ('active_ref_return', 'active_wap_return', 'active_mid_return', 'order_book_imbalance', 'auction_matched_pct')
            for halflife in (10, 20, 30, 60, 120)
        }
    )
)

df_ewm.drop(columns=['active_ref_return', 'active_wap_return', 'active_mid_return', 'order_book_imbalance', 'auction_matched_pct'], inplace=True)
df_ewm.to_parquet('model_data.gzip', compression='gzip')